In [52]:
"""
# 코랩에서는 이 셀 실행
import os
from google.colab import drive
drive.mount('/content/gdrive')

# 각자 구글 드라이브에 맞게 경로 수정 필요
path = "/content/gdrive/MyDrive/Colab Notebooks/pyspark/"
os.listdir(path)
"""

'\n# 코랩에서는 이 셀 실행\nimport os\nfrom google.colab import drive\ndrive.mount(\'/content/gdrive\')\n\n# 각자 구글 드라이브에 맞게 경로 수정 필요\npath = "/content/gdrive/MyDrive/Colab Notebooks/pyspark/"\nos.listdir(path)\n'

In [53]:
import pandas as pd
import numpy as np

In [54]:
path = "./" # local
ratings = pd.read_csv(path + "ratings_refined.csv", usecols=['userId', 'movieId', 'rating'])
ratings

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100784,610,166534,4.0
100785,610,168248,5.0
100786,610,168250,5.0
100787,610,168252,5.0


In [55]:
movies = pd.read_csv(path + "movies_refined.csv", usecols=['movieId', 'title'])
movies

,movieId,title
0,1,Toy Story
1,2,Jumanji
2,3,Grumpier Old Men
3,4,Waiting to Exhale
4,5,Father of the Bride Part II
...,...,...
9698,193581,Black Butler: Book of the Atlantic
9699,193583,No Game No Life: Zero
9700,193585,Flint
9701,193587,Bungo Stray Dogs: Dead Apple


In [56]:
df = pd.merge(ratings, movies, on='movieId', how='left')
df

,userId,movieId,rating,title
0,1,1,4.0,Toy Story
1,1,3,4.0,Grumpier Old Men
2,1,6,4.0,Heat
3,1,47,5.0,Seven (a.k.a. Se7en)
4,1,50,5.0,"Usual Suspects, The"
...,...,...,...,...
100784,610,166534,4.0,Split
100785,610,168248,5.0,John Wick: Chapter Two
100786,610,168250,5.0,Get Out
100787,610,168252,5.0,Logan


In [57]:
df.columns[df.isna().any()].tolist()

[]

# Item-based CF

## 영화 유사도 행렬 준비

In [58]:
movie_user = df.pivot_table(values='rating', index='title', columns='userId')
movie_user

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
'Hellboy': The Seeds of Creation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Round Midnight,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Salem's Lot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Til There Was You,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
eXistenZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.0,NaN,NaN,NaN,NaN,4.5,NaN,NaN
xXx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,2.0
xXx: State of the Union,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5


In [59]:
# 9413 x 610 행렬
# 영화를 610차원의 벡터로 보려는 것
movie_user.shape

(9413, 610)

## 결측치 처리

null값이 있으면 cosine similarity 함수가 안돌아감

하지만, null값을 0으로 치환하고 계산할경우 결과가 달라짐

(마치 해당 영화를 보고 0점을 준것으로 계산)

In [60]:
# 이 부분 KNN이나 다른 방법으로도 전처리 하기
movie_user_tmp = movie_user.copy().fillna(0)

## 유사도 행렬 계산

In [61]:
from sklearn.metrics.pairwise import cosine_similarity

movie_similarity_matrix = cosine_similarity(movie_user_tmp)
movie_similarity_matrix.shape

(9413, 9413)

In [62]:
movie_similarity_matrix

array([[1.        , 0.        , 0.        , ..., 0.32732684, 0.        ,
        0.        ],
       [0.        , 1.        , 0.70710678, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.70710678, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.32732684, 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

## 데이터 프레임화

In [63]:
movie_titles = movie_user.index
movie_titles

Index([''71', ''Hellboy': The Seeds of Creation', ''Round Midnight',
       ''Salem's Lot', ''Til There Was You', ''Tis the Season for Love',
       ''burbs, The', ''night Mother', '(500) Days of Summer',
       '*batteries not included',
       ...
       'Zulu', '[REC]', '[REC]²', '[REC]³ 3 Génesis',
       'anohana: The Flower We Saw That Day - The Movie', 'eXistenZ', 'xXx',
       'xXx: State of the Union', '¡Three Amigos!',
       'À nous la liberté (Freedom for Us)'],
      dtype='object', name='title', length=9413)

In [64]:
# 영화-영화 유저관람 내역에 대한 유사도
movie_similarity = pd.DataFrame(movie_similarity_matrix,
                                index=movie_titles, columns=movie_titles)
print(movie_similarity.shape)
movie_similarity.head()

(9413, 9413)


title,'71,'Hellboy': The Seeds of Creation,'Round Midnight,'Salem's Lot,'Til There Was You,'Tis the Season for Love,"'burbs, The",'night Mother,(500) Days of Summer,*batteries not included,...,Zulu,[REC],[REC]²,[REC]³ 3 Génesis,anohana: The Flower We Saw That Day - The Movie,eXistenZ,xXx,xXx: State of the Union,¡Three Amigos!,À nous la liberté (Freedom for Us)
title,,,,,,,,,,,,,,,,,,,,,
'71,1.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.141653,0.0,...,0.0,0.342055,0.543305,0.707107,0.0,0.0,0.139431,0.327327,0.0,0.0
'Hellboy': The Seeds of Creation,0.0,1.000000,0.707107,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
'Round Midnight,0.0,0.707107,1.000000,0.000000,0.000000,0.0,0.176777,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
'Salem's Lot,0.0,0.000000,0.000000,1.000000,0.857493,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
'Til There Was You,0.0,0.000000,0.000000,0.857493,1.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0


In [65]:
# timestamp 제거
print(ratings.index)
# ratings_ib = ratings.drop('timestamp', axis=1)
# movie ID와 title 빼고 다른 데이터 제거
# movies_ib = movies[['movie_id', 'title']]

RangeIndex(start=0, stop=100789, step=1)


In [66]:
from sklearn.model_selection import train_test_split

# train, test 데이터 분리
x = ratings.copy()
y = ratings['userId']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y, random_state=1)

In [67]:
# 정확도(RMSE)를 계산하는 함수
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

# 모델별 RMSE를 계산하는 함수
def score(model):
    id_pairs = zip(x_test['userId'], x_test['movieId'])
    y_pred = np.array([model(user, movie) for (user, movie) in id_pairs])
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

# 중복된 (userId, movieId) 조합 제거
x_train = x_train.drop_duplicates(subset=['userId', 'movieId'])

# train 데이터로 Full matrix 구하기
rating_matrix = x_train.pivot(index='userId', columns='movieId', values='rating')

In [68]:
#train set의 모든 가능한 아이템 pair의 Cosine simiLarities 계산
rating_matrix_t = np.transpose(rating_matrix)
matrix_dummy = rating_matrix_t.copy().fillna(0)
item_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
item_similarity = pd.DataFrame(item_similarity, index=rating_matrix_t.index, columns=rating_matrix_t.index)

In [73]:
# Cosine 기반 추천 함수
def Cosine_Dot_Production(user_id, movie_id):
    if movie_id in item_similarity:
        # 현재 영화가 train set에 있는지 확인
        # 현재 영화와 다른 영화의 Similarity 값 가져오기
        sim_scores = item_similarity[movie_id]
        # 현 사용자의 모든 rating 값 가져오기
        user_rating = rating_matrix_t[user_id]
        # 사용자가 평가하지 않은 영화 index 가져오기
        non_rating_idx = user_rating[user_rating.isnull()].index
        # 사용자가 평가하지 않은 영화 제거
        user_rating = user_rating.dropna()
        # 사용자가 평가하지 않은 영화의 similarity 값 제거
        sim_scores = sim_scores.drop(non_rating_idx)

        # 현 영화에 대한 예상 rating 계산
        # 분모가 0이 아닐 경우에만 계산
        if sim_scores.sum() > 0:
            mean_rating = np.dot(sim_scores, user_rating) / sim_scores.sum()
        else:
            # 분모가 0일 경우, 기본 rating 값 할당
            mean_rating = 3.0
    else:
        mean_rating = 3.0
    return mean_rating

score(Cosine_Dot_Production)

0.9183875951590599

In [ ]:
# SVD 기반 추천 함수
def SVD_Dot_Production(user_id, movie_id, k=50):
    # SVD 수행
    U, sigma, Vt = svds(rating_matrix, k=k)
    sigma = np.diag(sigma)

    # 사용자 예측 평점 계산
    user_predicted_ratings = np.dot(np.dot(U, sigma), Vt)
    user_predicted_ratings_df = pd.DataFrame(user_predicted_ratings, index=rating_matrix.index, columns=rating_matrix.columns)

    # 특정 사용자와 영화가 데이터프레임에 존재하는지 확인
    if user_id in user_predicted_ratings_df.index and movie_id in user_predicted_ratings_df.columns:
        # 특정 사용자에 대한 특정 영화의 예측 평점 반환
        return user_predicted_ratings_df.loc[user_id, movie_id]
    else:
        # 사용자 ID 또는 영화 ID가 없는 경우 기본 평점 반환
        return 3.0

score(SVD_Dot_Production)

In [ ]:
# 피어슨 상관계수 기반 추천 함수
def Pearson_Dot_Production(user_id, movie_id):
    # 피어슨 상관계수 계산
    rating_matrix_t = np.transpose(rating_matrix)
    similarity_matrix = rating_matrix_t.corr(method='pearson')

    # 특정 영화와 사용자가 데이터에 존재하는지 확인
    if movie_id in similarity_matrix.columns and user_id in rating_matrix_t.columns:
        sim_scores = similarity_matrix[movie_id]
        user_rating = rating_matrix_t[user_id]
        non_rating_idx = user_rating[user_rating.isnull()].index
        user_rating = user_rating.dropna()
        sim_scores = sim_scores.drop(non_rating_idx)

        # 현 영화에 대한 예상 평점 계산
        if sim_scores.sum() > 0:
            mean_rating = np.dot(sim_scores, user_rating) / sim_scores.sum()
        else:
            mean_rating = 3.0
    else:
        # 영화 ID 또는 사용자 ID가 없는 경우 기본 평점 반환
        mean_rating = 3.0

    return mean_rating


score(Pearson_Dot_Production)

In [ ]:
# Jaccard 유사도 기반 추천 함수
# 희소 데이터에 효과적일 수 있음
from sklearn.metrics import jaccard_score

def calculate_jaccard_similarity(rating_matrix):
    # 이진 형태로 변환 (평가 여부만 고려)
    binarized_matrix = rating_matrix.notnull().astype(int)
    similarity_matrix = np.zeros((binarized_matrix.shape[1], binarized_matrix.shape[1]))

    # Jaccard 유사도 계산
    for i in range(binarized_matrix.shape[1]):
        for j in range(binarized_matrix.shape[1]):
            if i != j:
                similarity_matrix[i, j] = jaccard_score(binarized_matrix.iloc[:, i], binarized_matrix.iloc[:, j])

    return pd.DataFrame(similarity_matrix, index=rating_matrix.columns, columns=rating_matrix.columns)

# Jaccard 유사도 기반 추천 함수
def Jaccard_Dot_Production(user_id, movie_id, rating_matrix):
    # Jaccard 유사도 계산
    jaccard_similarity = calculate_jaccard_similarity(rating_matrix)

    if movie_id in jaccard_similarity:
        sim_scores = jaccard_similarity[movie_id]
        user_rating = rating_matrix[user_id]
        non_rating_idx = user_rating[user_rating.isnull()].index
        user_rating = user_rating.dropna()
        sim_scores = sim_scores.drop(non_rating_idx)

        # 현 영화에 대한 예상 평점 계산
        if sim_scores.sum() > 0:
            mean_rating = np.dot(sim_scores, user_rating) / sim_scores.sum()
        else:
            mean_rating = 3.0
    else:
        mean_rating = 3.0

    return mean_rating

score(Jaccard_Dot_Production)